## Assignment 2 

Authors: Catherine Slaughter and Paloma Jol of Group 23  
Assignment 2 of the course introduction to machine learning (IML)  
Due: 23 december 2022 23:59


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import astropy
import astropy.units as u
import astropy.coordinates as coord

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

from sklearn import tree
from sklearn import ensemble

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

import seaborn as sns

Start pre-processing data, and exploring data analysis

In [ ]:
def read_in_data(filename = 'data/A2_data.csv'):
    '''Reads in the data from the given csv, and saves it in a Pandas dataframe'''
    df = pd.read_csv(filename)
    X,y= df.loc[:, df.columns != 'class'], df['class']
    return df, X, y

def train_test_split_drop(X,y, test_size : float, random_state=42, drop=True):
    '''Split the test and training data using a random state, if drop is True the index are reset after'''
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=test_size, random_state=42)
    X_train,y_train=X_train.reset_index(drop=drop),y_train.reset_index(drop=drop)
    X_test,y_test=X_test.reset_index(drop=drop),y_test.reset_index(drop=drop)
    return X_train, X_test, y_train, y_test

def histogram(column):
    '''Plot the histograms of the features'''
    plt.hist(X[column],facecolor='blue', alpha=0.8,histtype='bar', ec='black')
    plt.title(column)
    plt.xlabel('Feature label')
    plt.ylabel('Number of samples')
    plt.show()

def feature_importances_plot(forest, X_data, title = 'Feature Importances Using MDI', ymax = None):
    '''Plot random forest feature importances'''
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)

    feature_names = X_data.columns
    forest_importances = pd.Series(importances, index=feature_names)

    x_locs = range(importances.size)

    fig = plt.figure()
    ax = fig.add_subplot(111)

    forest_importances.plot.bar(yerr=std, ax=ax, capsize = 3)
    
    ax.set_title(title)
    ax.set_ylabel('Mean Decrease in Impurity')
    ax.set_ylim(0,ymax)
    return

In [ ]:
path = "data/"

data, X, y =read_in_data(path+'A2_data.csv')

print(f'There are {X.shape[0]} samples of each {X.shape[1]} features')

corr_data = X.copy()#for the later correlation plot

#Data includes some identifiers lets remove those 
ID_parameters= ['field_ID','MJD','plate','alpha', 'delta']
for ID in ID_parameters:
    X=X.loc[:, X.columns != ID]

histogram('u')

#We know that a flux should be positive so remove the datapoint which does not have that
#This datapoint has value -9999 so not a detection but a instrumentation issue
I_remove=np.where(X['u']<0)[0]
print(data.loc[I_remove])
X,y = X.drop(I_remove),y.drop(I_remove)
X,y=X.reset_index(drop=True),y.reset_index(drop=True)
data_processed= X.join(y)
data_processed.to_csv(path+'A2_data_preprocessed.csv', index=False)

corr_data = corr_data.drop(I_remove)
corr_data = corr_data.reset_index(drop=True)


print(f'There are final {X.shape[0]} samples of now {X.shape[1]} features')

X_train, X_test, y_train, y_test = train_test_split_drop(X,y, test_size=0.33, random_state=42, drop=True)


print(f'There are {X_train.shape[0]} training samples of each {X_train.shape[1]} features')

for C in X.columns:
    histogram(C)

## Sky-Map Visualization

In [ ]:
ra = coord.Angle(data['alpha']*u.degree)
ra = ra.wrap_at(180*u.degree)
dec = coord.Angle(data['delta']*u.degree)

new_df_arr = np.array([ra,dec,data['class']])
plot_df = pd.DataFrame(np.transpose(new_df_arr), columns = ['ra','dec','class'])

In [ ]:
fig = plt.figure(figsize=(8,12))
colors = ['red', 'green', 'blue']

for idx, cls in enumerate(['GALAXY', 'QSO', 'STAR']):
    loc = 311+idx
    ax = fig.add_subplot(loc, projection="mollweide")
    ra_sub = plot_df.loc[plot_df['class'] == cls, 'ra']*u.degree.to(u.rad)
    dec_sub = plot_df.loc[plot_df['class'] == cls, 'dec']*u.degree.to(u.rad)
    
    ax.scatter(ra_sub, dec_sub, s = 5, c=colors[idx])
    ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
    ax.grid(True)
    ax.set_title(cls)
    
plt.tight_layout()    
#plt.savefig('./figures/skymaps.pdf', dpi=300, bbox_inches = 'tight')

## Correlation Visualization

### Correlation Plot

In [ ]:
corr = corr_data.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(corr_data.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(corr_data.columns)
ax.set_yticklabels(corr_data.columns)
ax.set_title('Feature Correlation Plot', y = -0.1)
#plt.savefig('./figures/correlationplot.pdf', dpi=300, bbox_inches = 'tight')

### Pair Plot

_This takes a second to run!_

In [ ]:
pairplot_data = X.copy()
pairplot_data['class'] = y
color_pallete = {
    'GALAXY' : 'red',
    'QSO' : 'green',
    'STAR' : 'blue'
    
}

sns.pairplot(pairplot_data, hue = 'class', palette = color_pallete)
#plt.savefig('./figures/pairplot.pdf', dpi=300, bbox_inches = 'tight')

## Start dimensionality reductions

In [ ]:
#Visualise the dimension models
def visualise_components(model_name, y, finaldim, save=False):
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Component 1', fontsize = 15)
    ax.set_ylabel('Component 2', fontsize = 15)
    ax.set_title(f'2 component {model_name}', fontsize = 20)
    targets = np.unique(y)
    colors = ['r', 'g', 'b','k','y']
    for target, color in zip(targets,colors):
        indicesToKeep = finaldim['class'] == target
        ax.scatter(finaldim.loc[indicesToKeep, 'component 1']
                , finaldim.loc[indicesToKeep, 'component 2']
                , c = color
                , s = 10
                , marker = '.'
                , alpha=0.2)
    ax.legend(targets, fontsize=15)
    ax.grid()
    if save == True:
        plt.savefig(f'plots/{model_name}.pdf')
    plt.show()

In [ ]:
#Load in preprocessed data
path = "data/"
preprocessed, X, y = read_in_data(path+'A2_data_preprocessed.csv')
X_train, X_test, y_train, y_test = train_test_split_drop(X,y, test_size=0.33, random_state=42, drop=True) #random state chosen for reproducable results

x = X_train
x = StandardScaler().fit_transform(x)

params_list=[{
    "n_components": 2,
    "svd_solver": 'auto'
},{
     "n_components": 5,
    "svd_solver": 'auto'
}]


# Use PCA to reduce to two and to five dimensions for testing in RF
for params in params_list:
    pca = PCA(**params)
    c = params['n_components']
    principalComponents = pca.fit_transform(x)
    principal = pd.DataFrame(data = principalComponents,columns = np.array([f'component {i}' for i in range(1,c+1)],dtype=str))
    finalPca = pd.concat([principal, y_train], axis = 1)

    visualise_components('PCA', y_train, finalPca) #only the first two components are shown for the five components

    finalPca.to_csv(f'PCA_reduced_{c}_alphadelta.csv', index=False)

In [ ]:
#Experiments run on UMAP since this method had visaully the best results and was faster to run than TSNE
params_list=[{
    "n_neighbors": 2,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 5,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 50,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 100,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 200,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.0,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.1,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.25,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.4,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.45,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.5,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "euclidean", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "minkowski", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "mahalanobis", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "jaccard", 
    "init": "random"
},{
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "chebyshev", 
    "init": "random"
}]

In [ ]:
#UMAP
#best parameters from experiments
params_list=[{
    "n_components": 2,
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
},{
    "n_components": 5,
    "n_neighbors": 10,
    "min_dist": 0.3,
    "metric": "correlation", 
    "init": "random"
}]
for params in params_list:
    print(params['n_neighbors'])
    x = X_train
    x = StandardScaler().fit_transform(x)
    c = params['n_components']
    reducer = umap.UMAP(**params)
    UmapComponents = reducer.fit_transform(x)
    UmapDf = pd.DataFrame(data = UmapComponents,columns = np.array([f'component {i}' for i in range(1,c+1)],dtype=str))
    finalUmapDf = pd.concat([UmapDf, y_train], axis = 1)
    visualise_components(f"UMAP", y_train, finalUmapDf)
    finalUmapDf.to_csv(f'Umap_reduced_data_with_{c}.csv', index=False)

In [ ]:
#Some experiments run on T-SNE to see the effect of changing perplexity
params_list=[{
    'perplexity' : 5 ,
    'early_exaggeration' : 12.0 ,
    'metric': 'euclidean'
},{
    'perplexity' : 30 ,
    'early_exaggeration' : 12.0 ,
    'metric': 'euclidean'
},{
    'perplexity' : 50 ,
    'early_exaggeration' : 12.0 ,
    'metric': 'euclidean'
}]

# Use T-SNE to reduce to two dimensions
for params in params_list:
    x = X_train
    tsne = TSNE(n_components=2, verbose=1, perplexity=params['perplexity'], early_exaggeration=params['early_exaggeration'], metric=params['metric'])
    tsneComponents = tsne.fit_transform(x)
    tsneDf = pd.DataFrame(data = tsneComponents,columns = ['component 1', 'component 2'])
    finalTsneDf = pd.concat([tsneDf, y_train], axis = 1)
    finalTsneDf

    visualise_components(f"T-SNE_perplex{params['perplexity']}", y_train, finalTsneDf)

## Start clustering

In [ ]:
class Cluster_object():
    def __init__(self,method):
        self.method= method 

    def cluster_fit(self,X,params):
        clustering = self.method(**params)
        clustering.fit(X)
        return clustering

    def score(self,y_pred, y):
        nmis=0 #number misclassified
        for i in range(len(y_pred)):
            if y_pred[i] != y[i]:
                nmis+=1
        accuracy=nmis/len(y)
        print(f'Out of {len(y)} there are {nmis} missclassified samples, thus an accuracy of {accuracy}')
        return accuracy

In [ ]:
#Data import of dimension reduction
path='data_last/'
reduced_c2=pd.read_csv(path+'Umap_reduced_data.csv')
visualise_components('Umap', reduced_c2['class'], reduced_c2)

reduced_c5=pd.read_csv(path+'Umap_reduced_data_with_5.csv')
visualise_components('Umap', reduced_c5['class'], reduced_c5)

#reduced_pca=pd.read_csv(path+'PCA_reduced_5.csv')
#visualise_components('PCA', reduced_pca['class'], reduced_pca)


In [ ]:
#Clustering the pre-processed data
path = "data/"
preprocessed, X, y = read_in_data(path+'A2_data_preprocessed.csv')
X_train, X_test, y_train, y_test = train_test_split_drop(X,y, test_size=0.33, random_state=42, drop=True) #random state chosen for reproducable results
X_scale=scale(X_train)
Class_Names=np.unique(y_train)
Kmeans=Cluster_object(KMeans)
Kmean_cluster= Kmeans.cluster_fit(X_scale,params={'n_clusters': len(Class_Names), 'random_state':4})
y_pred=Class_Names[Kmean_cluster.labels_]
Kmeans_accuracy= Kmeans.score(y_pred,y_train)
labels=pd.DataFrame(y_pred,columns=['class'])
kmeans_Df=X.join(labels)


#Clustering the reduced data
reduced_c2, X, y= read_in_data(path+'Umap_reduced_data.csv')
X_scale=scale(X)
Class_Names=np.unique(y)
Kmeans=Cluster_object(KMeans)
Kmean_cluster= Kmeans.cluster_fit(X_scale,params={'n_clusters': len(Class_Names), 'random_state':1})
y_pred=Class_Names[Kmean_cluster.labels_]
Kmeans_accuracy= Kmeans.score(y_pred,y)
labels=pd.DataFrame(Kmean_cluster.labels_,columns=['class'])
kmeans_Df_reduced=X.join(labels)

visualise_components('UMAP',Class_Names, reduced_c2)

visualise_components('UMAP after K means', Kmean_cluster.labels_, kmeans_Df_reduced, save=True)


In [ ]:
params_list= [{'eps':  0.001 , 'min_samples':  50.0 }, 
{'eps':  0.001 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.001 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.001 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.001 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.001 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.001 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.001 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.001 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.001 , 'min_samples':  10000.0 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  50.0 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.0064444444444444445 , 'min_samples':  10000.0 }, 
{'eps':  0.01188888888888889 , 'min_samples':  50.0 }, 
{'eps':  0.01188888888888889 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.01188888888888889 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.01188888888888889 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.01188888888888889 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.01188888888888889 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.01188888888888889 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.01188888888888889 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.01188888888888889 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.01188888888888889 , 'min_samples':  10000.0 }, 
{'eps':  0.017333333333333333 , 'min_samples':  50.0 }, 
{'eps':  0.017333333333333333 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.017333333333333333 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.017333333333333333 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.017333333333333333 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.017333333333333333 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.017333333333333333 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.017333333333333333 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.017333333333333333 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.017333333333333333 , 'min_samples':  10000.0 }, 
{'eps':  0.02277777777777778 , 'min_samples':  50.0 }, 
{'eps':  0.02277777777777778 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.02277777777777778 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.02277777777777778 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.02277777777777778 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.02277777777777778 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.02277777777777778 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.02277777777777778 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.02277777777777778 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.02277777777777778 , 'min_samples':  10000.0 }, 
{'eps':  0.028222222222222225 , 'min_samples':  50.0 }, 
{'eps':  0.028222222222222225 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.028222222222222225 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.028222222222222225 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.028222222222222225 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.028222222222222225 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.028222222222222225 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.028222222222222225 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.028222222222222225 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.028222222222222225 , 'min_samples':  10000.0 }, 
{'eps':  0.033666666666666664 , 'min_samples':  50.0 }, 
{'eps':  0.033666666666666664 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.033666666666666664 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.033666666666666664 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.033666666666666664 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.033666666666666664 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.033666666666666664 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.033666666666666664 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.033666666666666664 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.033666666666666664 , 'min_samples':  10000.0 }, 
{'eps':  0.03911111111111111 , 'min_samples':  50.0 }, 
{'eps':  0.03911111111111111 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.03911111111111111 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.03911111111111111 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.03911111111111111 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.03911111111111111 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.03911111111111111 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.03911111111111111 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.03911111111111111 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.03911111111111111 , 'min_samples':  10000.0 }, 
{'eps':  0.04455555555555556 , 'min_samples':  50.0 }, 
{'eps':  0.04455555555555556 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.04455555555555556 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.04455555555555556 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.04455555555555556 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.04455555555555556 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.04455555555555556 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.04455555555555556 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.04455555555555556 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.04455555555555556 , 'min_samples':  10000.0 }, 
{'eps':  0.05 , 'min_samples':  50.0 }, 
{'eps':  0.05 , 'min_samples':  1155.5555555555557 }, 
{'eps':  0.05 , 'min_samples':  2261.1111111111113 }, 
{'eps':  0.05 , 'min_samples':  3366.666666666667 }, 
{'eps':  0.05 , 'min_samples':  4472.222222222223 }, 
{'eps':  0.05 , 'min_samples':  5577.777777777778 }, 
{'eps':  0.05 , 'min_samples':  6683.333333333334 }, 
{'eps':  0.05 , 'min_samples':  7788.88888888889 }, 
{'eps':  0.05 , 'min_samples':  8894.444444444445 }, 
{'eps':  0.05 , 'min_samples':  10000.0 } ]

In [ ]:
##DBSCAN
#best parameters
params_list=[{'eps': 0.05, 'min_samples': 50.0},
{'eps': 0.04455555555555556, 'min_samples': 50.0},
{'eps':0.2, 'min_samples': 1e4}]#last one illustrative gives only outliers as result

Dbscan=Cluster_object(DBSCAN) 

for params_db in params_list:
    Dbscan_cluster_reduced= Dbscan.cluster_fit(X_scale,params=params_db)
    #y_pred=Class_Names[Dbscan_cluster_reduced.labels_] #Since often we do not have only 3 clusters this will be out of bounds
    y_pred=Dbscan_cluster_reduced.labels_
    print(params_db, len(np.unique(y_pred)))
    
    #Dbscan_accuracy= Dbscan.score(y_pred,y) #Acuuracy harder to judge with unequal number of clusters
    labels_reduced=pd.DataFrame(y_pred,columns=['class'])
    dbscan_Df_reduced=X.join(labels_reduced)

    visualise_components('UMAP after dbscan',y_pred, dbscan_Df_reduced)


## Random Tree Cross-Validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split_drop(X,y)

In [ ]:
tree_clf = tree.DecisionTreeClassifier()
cv_output = cross_validate(tree_clf, X_train, y_train, cv=5, return_train_score=True)
print("Simple cross-validation yields average %0.5f accuracy +/- %0.5f" % (cv_output['test_score'].mean(), 
                                                                           cv_output['test_score'].std()))


### Create Plot

In [ ]:
maxdepth = np.arange(1, 21, 1)
#maxdepth[0]+=1

valmeans = []
valstds = []

trameans = []
trastds = []

for idx, depth in enumerate(maxdepth):
    clf_tree = tree.DecisionTreeClassifier(max_depth = depth)
    cv_output = cross_validate(clf_tree, X_train, y_train, cv=5, return_train_score=True)
    
    trameans.append(cv_output['train_score'].mean())
    trastds.append(cv_output['train_score'].std())   
    
    valmeans.append(cv_output['test_score'].mean())
    valstds.append(cv_output['test_score'].std()) 

In [ ]:
plt.errorbar(maxdepth, trameans, yerr=trastds, ecolor = 'black', capsize = 3, label = 'Training')
plt.errorbar(maxdepth, valmeans, yerr=valstds, ecolor = 'black', capsize = 3, label = 'Validation')
plt.xticks(maxdepth[1:])
plt.title('Max Depth vs. Accuracies')
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.legend();

#plt.savefig('./figures/trainvsvalidation.pdf', dpi=300, bbox_inches = 'tight')

## Hyper-parameter Tuning via Cross-Validation

### Read in data

In [ ]:
red_data=pd.read_csv(path+'PCA_reduced_5.csv') #read in data
red_X,red_y= red_data.loc[:, red_data.columns != 'class'], red_data['class']

#create training/test data for GSCV
print(f'There are total {red_X.shape[0]} samples of {red_X.shape[1]} features')
X_train, X_test, y_train, y_test = train_test_split_drop(red_X,red_y)
print(f'There are {X_train.shape[0]} training samples of each {X_train.shape[1]} features')

#### Create pair plot

In [ ]:
#creates a neat little pairplot of the data, useful for visualization, though takes a bit to run
pairplot_data = red_data
pairplot_data['class'] = y
color_pallete = {
    'GALAXY' : 'red',
    'QSO' : 'green',
    'STAR' : 'blue'
    
}

sns.pairplot(pairplot_data, hue = 'class', palette = color_pallete)
#plt.savefig('./figures/reducedpairplot.pdf', dpi=300, bbox_inches = 'tight')

### Run GSCV
_This takes a while!_

In [ ]:
n_features = 5 #number of features in the reduced data set

params = { #dictionary of possible parameters
    'criterion':  ['gini', 'entropy'],
    'max_depth':  [6,7,8,9,10],
    'max_features': np.linspace(1,n_features,3).astype(int).tolist(), #for a 5-feature dataset, this gives [1,3,5]
    'max_samples': [.25,.5,.75,1.],
    'n_estimators': [150]
}

forest_clf = GridSearchCV(estimator=ensemble.RandomForestClassifier(), param_grid=params, cv=5, n_jobs=5, verbose=1, 
                   return_train_score = True) #create gridsearchcv

forest_clf.fit(X_train, y_train)
forest_clf.best_params_ #output "best" parameters

In [ ]:
#pull out important info.
params_df = pd.DataFrame(forest_clf.cv_results_['params'])
validation_df = pd.DataFrame(forest_clf.cv_results_['mean_test_score'], columns=['Validation'])
training_df = pd.DataFrame(forest_clf.cv_results_['mean_train_score'], columns=['Training'])

#put into single dataframe
gscv_output = pd.concat([params_df,validation_df,training_df],axis=1)
#calculate percent difference
gscv_output['Percent Difference'] = (gscv_output['Training'] - gscv_output['Validation'])/gscv_output['Training']

In [ ]:
#save output dataframe, if wanted
gscv_output.to_csv('data/reducedcrosscor_out.csv')

### Select Parameter Set

In [ ]:
#bounds on the reduced hypothesis space
minsearch = 0
maxsearch = .05

#create histogram of percent differences
plt.hist(gscv_output['Percent Difference'], bins = 40);
#plt.axvline(minsearch, c='grey', linestyle = 'dashed')
plt.axvline(maxsearch, c='grey', linestyle = 'dashed', label = 'Max boundary')
plt.title('Histogram of Percent Differences')
plt.xlabel('Percent Difference')
plt.ylabel('Count')
plt.legend()

# plt.savefig('./figures/meanDiffHist.pdf', dpi=300, bbox_inches = 'tight')

In [ ]:
#choose best parameters from this space
selected_best = gscv_output.loc[(gscv_output['Percent Difference'] > minsearch) & (gscv_output['Percent Difference'] < maxsearch)].iloc[[gscv_output.loc[(gscv_output['Percent Difference'] > minsearch) & (gscv_output['Percent Difference'] < maxsearch), 'Validation'].argmax()]]
selected_idx = selected_best.index.item() #index needed for later visualization


In [ ]:
#create dictionary of selected parameters
selected_params = {'criterion': selected_best['criterion'].item(),
                   'max_depth': int(selected_best['max_depth'].item()),
                   'max_features': selected_best['max_features'].item(), 
                   'max_samples' : selected_best['max_samples'].item(),
                   'n_estimators': selected_best['n_estimators'].item()}

#create random forest
selected = ensemble.RandomForestClassifier(**selected_params)

selected_params

### Analysis

In [ ]:
#plot the training and validation scores by index for all 120 options

plt.plot(np.arange(len(gscv_output['Training'])), gscv_output['Training'], label = 'Training')
plt.plot(np.arange(len(gscv_output['Validation'])), gscv_output['Validation'], label = 'Validation')
#plt.plot(np.arange(len(gscv_output['Percent Difference'])), gscv_output['Percent Difference'], label = 'Percent Difference')

plt.axvline(selected_idx, c='darkgrey', linestyle = 'dashed', label = 'Selected Best Parameters')

plt.title('Training and Validation Mean Score for All Parameter Combinations')
plt.xlabel('Index')
plt.ylabel('Accuracy')
plt.ylim(.7,1.01)
plt.legend();

#plt.savefig('./figures/scoresLinePlot.pdf', dpi=300, bbox_inches = 'tight')


#### Averaged Parameter Plot

In [ ]:
#Create averaged parameter plot
fig = plt.figure(figsize=(14,8))

xlabels = ['gini','entropy']
x_axis_locs = np.arange(len(xlabels))
parameterlist = ['criterion', 'max_depth', 'max_features', 'max_samples']

for idx, param in enumerate(parameterlist):
    loc = 221+idx
    ax = fig.add_subplot(loc)
    
    tra_means = []
    tra_stds = []
    val_means = []
    val_stds = []
    for value in gscv_output[param].unique():
        tra_means.append(gscv_output[gscv_output[param] == value]['Training'].mean())
        tra_stds.append(gscv_output[gscv_output[param] == value]['Training'].std())
        val_means.append(gscv_output[gscv_output[param] == value]['Validation'].mean())
        val_stds.append(gscv_output[gscv_output[param] == value]['Validation'].std())
    
    if idx == 0:
        #ax.bar(x_axis_locs-.24, tra_means, width=.47, label = 'Training Score', yerr = tra_stds, capsize = 3)
        ax.bar(x_axis_locs, val_means, width=.99, label = 'Validation Score', yerr = val_stds, capsize = 3)
        ax.set_xticks(x_axis_locs, xlabels)
        ax.set_ylim(0.82,0.92)
    
    else:
        #ax.errorbar(gscv_output[param].unique(), tra_means, label = 'Training Score')#, yerr = tra_stds, capsize = 3,ecolor='black')
        ax.errorbar(gscv_output[param].unique(), val_means, label = 'Validation Score')#, yerr = val_stds, capsize = 3,ecolor='black')
        ax.set_xticks(gscv_output[param].unique())
        #ax.set_ylim(0.82,0.92)
        
    ax.set_xlabel('Value')
    ax.set_ylabel('Mean Accuracy')
    ax.set_title(param)
    #ax.legend()
    
plt.suptitle('Parameter Value vs. Validation Score')
plt.tight_layout()

#plt.savefig('./figures/GSCVout.pdf', dpi=300, bbox_inches = 'tight')



#### Feature Importances Plot

In [ ]:
selected.fit(red_X,red_y)
feature_importances_plot(selected, red_X, title = 'Feature Importances for Selected Parameters', ymax = 1)
# plt.savefig('./figures/FI_selected.png', dpi=300, bbox_inches = 'tight')


### Calculate Averages

In [ ]:
#removing n_estimators and max_samples from dictionary so we can make a single tree with the same other parameters
tree_params = selected_params.copy()
del tree_params["n_estimators"]
del tree_params["max_samples"]
tree_params

clf_tree = tree.DecisionTreeClassifier(**tree_params)

In [ ]:
n_runs = 5

selected_values = []
tree_values = []
    
for i in range(n_runs):
    X_train_avg, X_test_avg, y_train_avg, y_test_avg = train_test_split_drop(red_X,red_y)
#     print('split')
    selected.fit(X_train_avg, y_train_avg)
#     print('selected trained')
#     print('gscv trained')
    tree_clf.fit(X_train_avg, y_train_avg)
#     print('tree trained')
    
    selected_values.append(selected.score(X_test_avg, y_test_avg))
    tree_values.append(tree_clf.score(X_test_avg, y_test_avg))
    
    print('Run '+str(i+1)+' complete')

selected_values = np.asarray(selected_values)
tree_values = np.asarray(tree_values)

print("Selected Params: %0.5f accuracy +/- %0.5f over %d runs" % (selected_values.mean(), selected_values.std(), n_runs))
print("Single Tree Params: %0.5f accuracy +/- %0.5f over %d runs" % (tree_values.mean(), tree_values.std(), n_runs))

